In [212]:
import pandas as pd

import sys
sys.path.append('../../lib/')
import utils as au

In [231]:
audl = au.audl_data()
games = au.get_games().replace('Nashville Nightwatch','Nashville NightWatch').drop_duplicates(['Away Team', 'Away Team Score', 
                                                                                               'Date Location','Home Team', 
                                                                                               'Home Team Score', 'Matchup Link', 
                                                                                               'Year', 'Date', 'Time',
                                                                                               'City', 'Week'])
teams = au.get_teams()

In [232]:
audl['Date/Time'] = pd.to_datetime(audl['Date/Time'],format='%Y-%m-%d %H:%M')
games['Date'] = pd.to_datetime(games['Date'],format='%Y-%m-%d')

In [233]:
def check_name(df_in,home_away):
    if teams[teams.Teamname == df_in[f'{home_away} Team']].iloc[0].Inaugurated > df_in.Year:
#         print(df_in[f'{home_away} Team'],teams[teams.Teamname == df_in[f'{home_away} Team']].iloc[0].Inaugurated, df_in.Year)
        return teams[teams.Teamname == df_in[f'{home_away} Team']].iloc[0].Formerly
    return df_in[f'{home_away} Team']
games['Home Team'] = games.apply(check_name,home_away='Home', axis=1)
games['Away Team'] = games.apply(check_name,home_away='Away', axis=1)



In [234]:
foo = audl.groupby(['Year','Teamname','GameID']).apply(lambda x : x['Date/Time'].nunique()).reset_index()
foo[foo[0]>1] # hooray. Games only have 1 Date/Time

,Year,Teamname,GameID,0


In [235]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

def closest_uni_game(df_in):
    teamname = df_in['Teamname'].unique()[0]
    opponent = df_in['Opponent'].unique()[0]
    dt = df_in['Date/Time'].dt.strftime('%Y-%m-%d').unique()[0]
    team_games = games[games['Away Team'].isin([teamname,opponent]) &
                       games['Home Team'].isin([teamname,opponent])].reset_index().copy()
    match = team_games.iloc[(team_games['Date'] - pd.to_datetime(dt)).abs().argsort()[0]]
    
    return pd.DataFrame( [{'UniversalGameID': match['UniversalGameID'] ,
                          'DaysOff':(team_games['Date'] - pd.to_datetime(dt)).abs().min()}])
    

unis = audl.groupby(['Year','Teamname','Opponent','GameID','Date/Time']).apply(closest_uni_game).reset_index()

In [236]:
unis = unis.drop('level_5',axis=1)

In [237]:
games[games.UniversalGameID==889]

,Unnamed: 0,Away Team,Away Team Score,Date Location,Home Team,Home Team Score,Matchup Link,Year,Date,Time,City,Week,UniversalGameID
889,889,Indianapolis AlleyCats,25,"Sat, 4/28 7:30 PM CDT Chicago",Chicago Wildfire,21,/league/game/2018-04-28-IND-CHI,2018,2018-04-28,7:30 PM CDT,Chicago,5,889


In [238]:
unis[unis['UniversalGameID'].apply(lambda x : len(unis[unis.UniversalGameID==x]))!=2].sort_values('UniversalGameID')

,Year,Teamname,Opponent,GameID,Date/Time,DaysOff,UniversalGameID
175,2014,San Francisco FlameThrowers,Salt Lake Lions,6,2014-05-10 17:42:00,21 days,181
161,2014,Salt Lake Lions,San Francisco FlameThrowers,3,2014-04-19 13:56:00,0 days,181
162,2014,Salt Lake Lions,San Francisco FlameThrowers,6,2014-05-10 17:45:00,21 days,181
174,2014,San Francisco FlameThrowers,Salt Lake Lions,2,2014-04-19 12:49:00,0 days,181
184,2014,San Francisco FlameThrowers,Seattle Raptors,5,2014-05-04 12:00:00,0 days,200
183,2014,San Francisco FlameThrowers,Seattle Raptors,3,2014-04-26 16:07:00,8 days,200
209,2014,Seattle Raptors,San Francisco FlameThrowers,2,2014-04-26 16:18:00,8 days,200
210,2014,Seattle Raptors,San Francisco FlameThrowers,4,2014-05-04 13:07:00,0 days,200
206,2014,Seattle Raptors,Salt Lake Lions,6,2014-05-18 11:53:00,0 days,218
45,2014,Detroit Mechanix,Chicago Wildfire,9,2014-05-31 18:10:00,0 days,228


In [239]:
games[((games.Date<'2019-05-01'))&(games.Date>'2014')&~games.UniversalGameID.isin(unis.UniversalGameID.unique())]

,Unnamed: 0,Away Team,Away Team Score,Date Location,Home Team,Home Team Score,Matchup Link,Year,Date,Time,City,Week,UniversalGameID
669,669,Seattle Cascades,27,"Sun, 8/7 12:00 PM CDT Dallas",Dallas Roughnecks,33,/league/game/2016-08-07-SEA-DAL,2016,2016-08-07,12:00 PM CDT,Dallas,19,669
842,842,Los Angeles Aviators,33,"Sat, 8/12 7:00 PM CDT San Jose",San Jose Spiders,24,/league/game/2017-08-12-LA-SJ,2017,2017-08-12,7:00 PM CDT,San Jose,20,843


In [242]:
games[games[['Away Team', 'Away Team Score', 'Date Location',
       'Home Team', 'Home Team Score', 'Matchup Link', 'Year', 'Date', 'Time',
       'City', 'Week']].duplicated()]

,Unnamed: 0,Away Team,Away Team Score,Date Location,Home Team,Home Team Score,Matchup Link,Year,Date,Time,City,Week,UniversalGameID
